In [1]:
import os
import cv2
import numpy as np
from sklearn import cluster
import matplotlib.pyplot as plt

In [2]:
path='./pictures/'

In [3]:
images = []

for img in os.listdir(path):
    if img.startswith('zzz'):
        image = cv2.imread(path+img)
        cv2.imshow('crop',image)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.waitKey(0)
        images.append(image)

cv2.destroyAllWindows()
len(images)

1

In [4]:

def segment(image, k):
    # reshape the image to a 2D array of pixels and 3 color values (RGB)
    pixel_values = image.reshape((-1, 3))
    # convert to float
    pixel_values = np.float32(pixel_values)

    print(pixel_values.shape)

    # define stopping criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)

    # number of clusters (K)
    _, labels, (centers) = cv2.kmeans(pixel_values, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # convert back to 8 bit values
    centers = np.uint8(centers)

    # flatten the labels array
    labels = labels.flatten()

    # convert all pixels to the color of the centroids
    segmented_image = centers[labels.flatten()]

    # reshape back to the original image dimension
    segmented_image = segmented_image.reshape(image.shape)
    # show the image
    cv2.imshow('segm', segmented_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    return labels

k=5
labels = segment(images[0], k)

(512000, 3)


In [5]:
def extract_region(image, regions, labels):
    for i in range(regions):

        # disable only the cluster number 2 (turn the pixel into black)
        masked_image = np.copy(image)

        # convert to the shape of a vector of pixel values
        masked_image = masked_image.reshape((-1, 3))
    
        # color (i.e cluster) to disable
        clust = i
        masked_image[labels != clust] = [255,255,255]

        # convert back to original shape
        masked_image = masked_image.reshape(image.shape)

        # make background transparent by placing the mask into the alpha channel
        mask = labels == clust
        mask = mask.reshape(masked_image.shape[:2])*255
        new_img = cv2.cvtColor(masked_image, cv2.COLOR_BGR2BGRA)
        new_img[:, :, 3] = mask
        cv2.imshow('crop',new_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        cv2.imwrite(f'{path}y_shirt_skirt_outfir_{i}.png', new_img)

    return

extract_region(images[0], k, labels)
